# Sensor Dashboard V0

This gives us a relatively easy way to inspect stored sensory data from any time window. Real-time inspection will  probably be questionable. But this should give us a good starting point. Just change the parameters and run the whole notebook.

In [18]:
from desmond import types
import os
import time

In [10]:
# Parameters
params = {}
params['start_time'] = None
params['end_time'] = None
params['data_types'] = (types.Text, types.Image)
params['database'] = os.path.join(os.path.expanduser('~'), '.desmond/sensorlogs.db')

In [113]:
import sqlite3

def micros(seconds):
    return int(seconds*1e6)

class SensorReader(object):
    def __init__(self, dbname):
        self.conn = sqlite3.connect(dbname)
        self.conn.row_factory = sqlite3.Row
        
    def read(self, start, end, datatype=None):
        args = micros(start), micros(end)
        query = ("select *from sensordata "
                 "where time_usec > ? and time_usec < ?")
        if datatype is not None:
            if '/' not in datatype:
                datatype = "type.googleapis.com/"+datatype
            args += (datatype,)
            query += " and type_url = ?"

        query += " order by time_usec desc"
            
        cur = self.conn.cursor()
        cur.execute(query, args)
        for elem in cur:
            yield elem
            
    def __del__(self):
        self.conn.close()

In [114]:
r = SensorReader(params['database'])

In [121]:
# Visualization, exporting utils
from IPython.display import HTML, display
from datetime import datetime
from google.protobuf import any_pb2

def html_table(data, keys=None):
    """ Returns HTML string for a table of the errors.
    
    Args:
        errors: List of dictionaries. All elements of dictionaries will be represented
                as strings in the table.
        keys: Keys from the dictionaries that should be shown. If None, will use keys
              from first element of the |errors|.

    Returns:
        Valid HTML string.
    """
    if not data:
        return ""

    keys = keys or data[0].keys()
    s = "<table>"
    s += "<tr><th>" + "</th><th>".join(keys) + "</th></tr>"
    row_template = "<tr><td>"+"</td><td>".join("{"+key+"}" for key in keys) + "</td></tr>"
    s += "".join(row_template.format(**row_data)
                                     for row_data in data)
    s += "</table>"
    return s

def show_table(data, keys=None):
    display(HTML(html_table(data, keys=keys)))
    
# Basic idea: create a map from type url to special decoder (e.g. for images)
def default_decode(payload):
    if (len(payload) < 40):
        return payload
    else:
        return payload[0:40]+"..."
    
def decode_text(payload):
    text = types.Text()
    text.ParseFromString(payload)
    return text.value
    
class TableViewer(object):
    DECODERS = {
        "desmond.types.Text": decode_text
    }
    """Default viewer for a sensory data type. Shows a table. """
    def __init__(self, DType=None, transforms=None):
        self.DType = DType
        self.transforms = {}
        self.transforms['time_usec'] = lambda time_usec: str(datetime.fromtimestamp(time_usec/1e6))
        self.transforms['payload'] = self.decode_payload
        if transforms:
            self.transforms.update(transforms)
        
    def decode_payload(self, type_url, value_bytes):
        decode_fn = TableViewer.DECODERS.get(type_url.split('/')[1], default_decode)
        return decode_fn(value_bytes)
    
    def display(self, rows):
        data = []
        for row in rows:
            row_data = dict((k, row[k]) for k in row.keys())
            row_data['timestamp'] = str(datetime.fromtimestamp(row['time_usec']/1e6))
            row_data.pop('time_usec')
            row_data['payload'] = self.decode_payload(row['type_url'], row['payload'])
            data.append(row_data)
            
        show_table(data, keys=('timestamp', 'payload', 'type_url', 'sensor_name', 'sensor_address'))
        

In [122]:
viewer = TableViewer()
viewer.display(r.read(0, time.time(), datatype="desmond.types.Text"))

timestamp,payload,type_url,sensor_name,sensor_address
2017-11-30 22:30:32.544773,may I have,type.googleapis.com/desmond.types.Text,Commandline,tcp://192.168.1.24:8979
2017-11-30 22:30:30.423300,sir,type.googleapis.com/desmond.types.Text,Commandline,tcp://192.168.1.24:8979
2017-11-30 22:30:29.697413,please,type.googleapis.com/desmond.types.Text,Commandline,tcp://192.168.1.24:8979
2017-11-30 22:30:27.708600,with these logs,type.googleapis.com/desmond.types.Text,Commandline,tcp://192.168.1.24:8979
2017-11-30 22:30:23.820875,what gives,type.googleapis.com/desmond.types.Text,Commandline,tcp://192.168.1.24:8979
2017-11-30 22:30:21.347047,yo,type.googleapis.com/desmond.types.Text,Commandline,tcp://192.168.1.24:8979
